In [3]:
import pandas as pd
!pip install transformers
from transformers import pipeline
# !pip install bert-embedding

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 84.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 19.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 35.3 MB/s eta 0:00:00


In [4]:
df = pd.read_csv('cleaned-dataset.tsv', sep = '\t')

In [5]:
df.head()

,Unnamed: 0,Date,HomeTeam,AwayTeam,FTHG,FTAG,FTR,HTGS,ATGS,HTGC,...,IWD,IWA,LBH,LBD,LBA,WHH,WHD,WHA,Commentary,Season
0,0,02/2014,Arsenal,Sunderland,4,1,H,48,25,26,...,5.2,10.00,1.30,5.5,10.00,1.33,4.5,11.00,Gus Poyet says the excitement at Sunderland ha...,13/14
1,1,02/2014,Chelsea,Everton,1,0,H,48,37,21,...,4.0,6.10,1.57,4.0,6.00,1.60,3.4,7.00,Before a reunion with Didier Drogba in midweek...,13/14
2,2,02/2014,Crystal Palace,Manchester United,0,2,NH,18,41,36,...,3.6,1.65,6.00,4.0,1.57,6.00,3.6,1.62,This promises to be an awkward test for Manche...,13/14
3,3,02/2014,Manchester City,Stoke City,1,0,H,69,27,27,...,7.0,15.00,1.18,7.0,15.00,1.20,6.0,15.00,This visit of Stoke City is the last league ga...,13/14
4,4,02/2014,West Brom,Fulham,1,1,NH,30,26,38,...,3.5,4.20,1.80,3.6,4.33,1.80,3.5,4.50,Felix Magath has always been known as a tough ...,13/14


Sentence classification

In [6]:
classifier = pipeline("zero-shot-classification", device = 0 )

No model was supplied, defaulted to facebook/bart-large-mnli and revision c626438 (https://huggingface.co/facebook/bart-large-mnli).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [ ]:
pre = df['Commentary'][3].split('.')

In [ ]:
pre

In [ ]:
sequence =  pre[0] 
candidate_labels = ["Manchester City", "Stoke City"]

classifier(sequence, candidate_labels)

{'sequence': 'Gus Poyet says the excitement at Sunderland has been reflected in the best week of training under his charge',
 'labels': ['Stoke City', 'Manchester City'],
 'scores': [0.5040212273597717, 0.49597880244255066]}

In [ ]:
home = []
away = []
for i in range(len(pre)-1): #last sentence == name of commentator 
  sequence =  pre[i]
  candidate_labels = ["Manchester City", "Stoke City"]
  score = classifier(sequence, candidate_labels)
  if score['scores'][0] > 0.6:
    if score['labels'][0] == "Manchester City":
      home.append(pre[i])
    else:
      away.append(pre[i])



In [125]:
home

0

In [ ]:
away

[' Despite the midweek loss to Barcelona in the Champions League, Pellegrini is confident his players will not be affected so Stoke may be in for a torrid time at the Etihad, where only Chelsea have taken any points from the home side']

Sentiment Analysis

In [ ]:
sentiment_pipeline = pipeline("sentiment-analysis", model="finiteautomata/bertweet-base-sentiment-analysis")
data_home = '.'.join(home)
sen_home = sentiment_pipeline(data_home)

emoji is not installed, thus not converting emoticons or emojis into text. Install emoji: pip3 install emoji==0.6.0


In [ ]:
data_away = '.'.join(away)
sen_away = sentiment_pipeline(data_away)

In [ ]:
sen_home

[{'label': 'POS', 'score': 0.7897095680236816}]

In [ ]:
sen_away

[{'label': 'NEU', 'score': 0.7743259072303772}]

Vectorisation


In [4]:
# BERT APPROACH
!pip install tensorflow-text

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 97.7 MB/s eta 0:00:00


In [5]:
import tensorflow_hub as hub
import tensorflow_text as text

In [6]:
preprocess_url = "https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3"
encoder_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4"

In [7]:
bert_preprocess_model = hub.KerasLayer(preprocess_url)

In [ ]:
text_test = home
text_preprocessed = bert_preprocess_model(text_test)
text_preprocessed.keys()

dict_keys(['input_type_ids', 'input_word_ids', 'input_mask'])

In [ ]:
bert_model = hub.KerasLayer(encoder_url)

In [ ]:
bert_results = bert_model(text_preprocessed)
bert_results.keys()

dict_keys(['pooled_output', 'encoder_outputs', 'sequence_output', 'default'])

In [ ]:
bert_results['sequence_output']

<tf.Tensor: shape=(1, 128, 768), dtype=float32, numpy=
array([[[-0.42693415, -0.01844354,  0.284892  , ..., -0.3737539 ,
          0.682148  ,  0.6948585 ],
        [-0.707907  , -0.97364706,  0.0861766 , ..., -0.28266454,
          1.0088437 ,  0.4399444 ],
        [-0.49876568, -0.1912384 ,  1.1104642 , ...,  0.4054509 ,
          0.69123375,  0.00943614],
        ...,
        [-0.5118868 , -0.51120424,  0.42049408, ...,  0.38659212,
          0.5172696 , -0.08234709],
        [-0.55293554, -0.22017416,  0.8257847 , ...,  0.1980631 ,
          0.25715527,  0.2296115 ],
        [-0.58896804, -0.29525095,  0.82477856, ...,  0.32399544,
          0.28103212,  0.1618569 ]]], dtype=float32)>

Splitting each preview into home and away sentences 

In [9]:
all = []
for j in range(len(df.index)):
  pre = df['Commentary'][j].split('.')
  home = []
  away = []
  for i in range(len(pre)-1): #last sentence == name of commentator 
    sequence =  pre[i]
    if pre[i] == '':
      continue
    candidate_labels = [df['HomeTeam'][j], df['AwayTeam'][j]]
    score = classifier(sequence, candidate_labels)
    if score['scores'][0] > 0.6:
      if score['labels'][0] == df['HomeTeam'][j]:
        home.append(pre[i])
      else:
        away.append(pre[i])
  all.append('.'.join(home))
  all.append('.'.join(away))

In [10]:
len(all)

2496

Using CountVectorizer Approach

In [27]:
from sklearn.feature_extraction.text import CountVectorizer

In [28]:
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(all)


In [29]:
home_vecs = X.toarray()[::2]
away_vecs = X.toarray()[1::2]

In [31]:
home_vecs.shape

(1248, 6578)

NEXT TASKS

1) Calculate alpha values

2) Concant alphas@home_vecs with away_vecs to form data/feature matrix (X).

3) Apply a multi-class Logistic Regression to find important features in X

4) Fit a random forest using data only from important features of X. Use grid search for hyperparameter tuning. 
